In [1]:
import librosa
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
from random import shuffle
from tqdm.notebook import tqdm
import pandas as pd
from time import time

import pickle

from scipy.special import softmax
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DATA_DIR=r'/content/drive/MyDrive/data/AudioVideo'
IMG_SIZE = 224

num_labels = 6 
emotions_list = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
class_to_idx = {'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happy': 3, 'Neutral': 4, 'Sad': 5, 'Surprise': 6}
idx_to_class = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}

# FP16 Ensemble OpenVINO

In [5]:
!pip3 install pickle5
path_to_protocol5 = '/content/drive/MyDrive/data/enet_b0_8_FP16_afew.pickle'

import pickle5 as p
import pickle


with open(path_to_protocol5, "rb") as fh:
  filename2features_train, filename2features_val = p.load(fh)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
def create_dataset(filename2features,data_dir):
    x = []
    y = []
    has_faces=[]
    ind=0
    for class_name in class_to_idx:
        for filename in tqdm(os.listdir(os.path.join(data_dir,class_name))):
            fn=os.path.splitext(filename)[0] # goes through files names
            if not fn in filename2features:
                continue
            features=filename2features[fn]
            total_features=None
            #print(len(features))
            if True:
                if len(features[0])!=0:
                    cur_features=features[0][features[-1]==1]
                #print(prev,features.shape)
            else:
                cur_features=features[0]
            if len(cur_features)==0:
                has_faces.append(0)
                total_features=np.zeros_like(feature)
            else:
                has_faces.append(1)
                #mean_features=features.mean(axis=0)
                mean_features = (np.mean(cur_features, axis=0))
                std_features = (np.std(cur_features, axis=0))
                max_features = (np.max(cur_features, axis=0))
                min_features = (np.min(cur_features, axis=0))

                # join several features together
                feature = np.concatenate((mean_features, std_features, min_features, max_features), axis=None)                    
                #feature = np.concatenate((mean_features, std_features, min_features), axis=None)
                #feature = np.concatenate((mean_features, min_features, max_features), axis=None)
                #feature = np.concatenate((max_features, std_features), axis=None)
                #feature=max_features

                total_features=feature
            
            if total_features is not None:
                x.append(total_features)
                y.append(class_to_idx[class_name])
    x=np.array(x)
    y=np.array(y)
    has_faces=np.array(has_faces)
    print(x.shape,y.shape)
    return x,y,has_faces

x_train_enet, y_train_enet, has_faces_train = create_dataset(filename2features_train, os.path.join(DATA_DIR, 'Train_AFEW'))
x_test_enet, y_test_enet, has_faces_test = create_dataset(filename2features_val, os.path.join(DATA_DIR, 'Val_AFEW'))

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

...

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

(383, 5120) (383,)


In [7]:
from sklearn import svm,metrics,preprocessing

x_train_norm=preprocessing.normalize(x_train_enet,norm='l2')
x_test_norm=preprocessing.normalize(x_test_enet,norm='l2')

In [38]:
lin_svc = svm.LinearSVC(C=1.1) #0.5 1.1 0.6
   
lin_svc.fit(x_train_norm[has_faces_train==1], y_train_enet[has_faces_train==1])
y_pred = lin_svc.predict(x_test_norm)

print("Accuracy:",metrics.accuracy_score(y_test_enet[has_faces_test==1], y_pred[has_faces_test==1]))
print("Complete accuracy:",metrics.accuracy_score(y_test_enet, y_pred))

Accuracy: 0.5926892950391645
Complete accuracy: 0.5926892950391645


In [39]:
linsvc_proba = []
linsvc_dist = lin_svc.decision_function(x_test_norm)
for i in range(len(linsvc_dist)):
  linsvc_proba.append(softmax(linsvc_dist[i]))

linsvc_proba = np.array(linsvc_proba)


## Audio

In [17]:
!pip install openvino-dev[EXTRAS]
!pip install transformers==4.11.3
!pip install openvino

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 11.3 MB/s 
     |████████████████████████████████| 3.1 MB 52.9 MB/s 
     |████████████████████████████████| 1.9 MB 54.9 MB/s 
     |████████████████████████████████| 1.2 MB 48.2 MB/s 
     |████████████████████████████████| 15.7 MB 51.7 MB/s 
     |████████████████████████████████| 25.9 MB 11.5 MB/s 
     |████████████████████████████████| 2.0 MB 56.2 MB/s 
     |████████████████████████████████| 14.8 MB 50.5 MB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 14.1 MB 21.3 MB/s 
     |████████████████████████████████| 9.5 MB 33.6 MB/s 
     |████████████████████████████████| 60.5 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 11.8 MB/s 
     |████████████████████████████████| 1.5 MB 51.1 MB/s 
     |████████████████████████████████| 603 kB 63.4 MB/s 
     |██████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import torchaudio
import torch
import numpy as np
from transformers import AutoConfig, Wav2Vec2Processor, AutoModelForAudioClassification

In [12]:
model_name_or_path = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
pooling_mode = "mean"
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
target_sampling_rate = processor.feature_extractor.sampling_rate

Downloading:   0%|          | 0.00/262 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

In [13]:
def preprocess_function_eval(speech_path):   
    speech_array, sampling_rate = librosa.load(speech_path, sr = 16000)
    result = processor(speech_array, sampling_rate=target_sampling_rate, max_length=50000, padding=True, truncation=True, return_attention_mask=True)
    len_of_input_data = result['input_values'][0].shape[0]
    padded_array = np.pad(result['input_values'][0], ((0,50000-len_of_input_data)), constant_values=0)
    return padded_array

In [14]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id=class_to_idx,
    id2label=idx_to_class,
    finetuning_task="wav2vec2_clf",    
)
setattr(config, 'pooling_mode', pooling_mode)

In [15]:
model = AutoModelForAudioClassification.from_pretrained(
    '/content/drive/MyDrive/models/wav2vec2/hugging_face_w2v2', # <- insert folder with xml, config and bin
    num_labels=6,
    label2id=class_to_idx,
    id2label=idx_to_class,
) # acc 0.415144


In [18]:
from openvino.runtime import Core

ie = Core()
classification_model_xml = "/content/drive/MyDrive/models/wav2vec2/fp16_w2v2/wav2vec2.xml" #<- insert here xml file where folder contains .bin weight file
model = ie.read_model(model=classification_model_xml)
model.reshape([1, 50000])
compiled_model = ie.compile_model(model=model, device_name="CPU") 
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

In [31]:
import pathlib 

path = pathlib.Path('/content/drive/MyDrive/data/AudioVideo/Val_AFEW_audio')

emotions_list = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
audio_proba = [] 

for emotion_name in emotions_list: 
  print(emotion_name)
  audio_path = path/emotion_name
  for audio in audio_path.iterdir():
      output = compiled_model([np.expand_dims(preprocess_function_eval(audio), 0)])[output_layer]
      audio_proba.append(softmax(output[0]))
      #input = torch.from_numpy(np.expand_dims(preprocess_function_eval(audio), 0)).to(device)
      #output = model(input)
      #audio_proba.append(softmax(output[0][0].cpu().numpy()))
audio_proba = np.array(audio_proba)

Angry
Disgust
Fear
Happy
Neutral
Sad
Surprise


In [35]:
import pickle
model_name = 'w2v2'
MODEL2EMOTIW_FEATURES=model_name+'_prob_audio_afew.pickle' 

print(MODEL2EMOTIW_FEATURES)

with open(MODEL2EMOTIW_FEATURES, 'wb') as handle:
    pickle.dump(audio_proba, handle, protocol=pickle.HIGHEST_PROTOCOL)

w2v2_prob_audio_afew.pickle


## Soft Voting

In [ ]:
path_to_audio_proba = '/content/drive/MyDrive/data/w2v2_prob_audio_afew.pickle'

with open(path_to_audio_proba, "rb") as fh:
  audio_proba = p.load(fh)

In [40]:
weights = np.arange(0, 1, 0.01)

best_weights = [0, 0]
best_acc = 0


for a in weights:
    for l in weights:
        y_pred = []
        for i in range(0, len(linsvc_proba)):
            max_prob = np.argmax(a*audio_proba[i]+l*linsvc_proba[i])
            y_pred.append(max_prob)
        if metrics.accuracy_score(y_test_enet, np.array(y_pred)) > best_acc:
            best_acc = metrics.accuracy_score(y_test_enet, np.array(y_pred))
            best_weights[0] = a
            best_weights[1] = l
            print(best_acc, best_weights)

0.1671018276762402 [0.0, 0.0]
0.5926892950391645 [0.0, 0.01]
0.6135770234986945 [0.01, 0.01]
0.6214099216710183 [0.02, 0.01]
0.6240208877284595 [0.05, 0.02]
0.6266318537859008 [0.05, 0.03]
0.6318537859007833 [0.08, 0.03]
0.6344647519582245 [0.13, 0.05]


In [41]:
best_weights = [0, 0, 0.13, 0.05]

y_pred = []

for i in range(0, len(linsvc_proba)):
  y_pred.append(np.argmax(best_weights[2]*audio_proba[i]+best_weights[3]*linsvc_proba[i]))

print("Complete accuracy:",metrics.accuracy_score(y_test_enet, np.array(y_pred)))

Complete accuracy: 0.6344647519582245


# FP32 Ensemble ONNX

In [4]:
!pip3 install pickle5
path_to_protocol5 = '/content/drive/MyDrive/data/enet_b0_8_afew_onnx.pickle'

import pickle5 as p
import pickle


with open(path_to_protocol5, "rb") as fh:
  filename2features_train, filename2features_val = p.load(fh)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 7.3 MB/s 


In [5]:
def create_dataset(filename2features,data_dir):
    x = []
    y = []
    has_faces=[]
    ind=0
    for class_name in class_to_idx:
        for filename in tqdm(os.listdir(os.path.join(data_dir,class_name))):
            fn=os.path.splitext(filename)[0] # goes through files names
            if not fn in filename2features:
                continue
            features=filename2features[fn]
            total_features=None
            #print(len(features))
            if True:
                if len(features[0])!=0:
                    cur_features=features[0][features[-1]==1]
                #print(prev,features.shape)
            else:
                cur_features=features[0]
            if len(cur_features)==0:
                has_faces.append(0)
                total_features=np.zeros_like(feature)
            else:
                has_faces.append(1)
                #mean_features=features.mean(axis=0)
                mean_features = (np.mean(cur_features, axis=0))
                std_features = (np.std(cur_features, axis=0))
                max_features = (np.max(cur_features, axis=0))
                min_features = (np.min(cur_features, axis=0))

                # join several features together
                feature = np.concatenate((mean_features, std_features, min_features, max_features), axis=None)                    
                #feature = np.concatenate((mean_features, std_features, min_features), axis=None)
                #feature = np.concatenate((mean_features, min_features, max_features), axis=None)
                #feature = np.concatenate((max_features, std_features), axis=None)
                #feature=max_features

                total_features=feature
            
            if total_features is not None:
                x.append(total_features)
                y.append(class_to_idx[class_name])
    x=np.array(x)
    y=np.array(y)
    has_faces=np.array(has_faces)
    print(x.shape,y.shape)
    return x,y,has_faces

x_train_enet, y_train_enet, has_faces_train = create_dataset(filename2features_train, os.path.join(DATA_DIR, 'Train_AFEW'))
x_test_enet, y_test_enet, has_faces_test = create_dataset(filename2features_val, os.path.join(DATA_DIR, 'Val_AFEW'))

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

...

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

(383, 5120) (383,)


In [6]:
from sklearn import svm,metrics,preprocessing

x_train_norm=preprocessing.normalize(x_train_enet,norm='l2')
x_test_norm=preprocessing.normalize(x_test_enet,norm='l2')

In [24]:
lin_svc = svm.LinearSVC(C=1.1) #0.5 1.1 0.6
   
lin_svc.fit(x_train_norm[has_faces_train==1], y_train_enet[has_faces_train==1])
y_pred = lin_svc.predict(x_test_norm)

print("Accuracy:",metrics.accuracy_score(y_test_enet[has_faces_test==1], y_pred[has_faces_test==1]))
print("Complete accuracy:",metrics.accuracy_score(y_test_enet, y_pred))

Accuracy: 0.5926892950391645
Complete accuracy: 0.5926892950391645


In [25]:
linsvc_proba = []
linsvc_dist = lin_svc.decision_function(x_test_norm)
for i in range(len(linsvc_dist)):
  linsvc_proba.append(softmax(linsvc_dist[i]))

linsvc_proba = np.array(linsvc_proba)


## Audio

In [9]:
!pip install openvino-dev[EXTRAS]
!pip install transformers==4.11.3
!pip install openvino

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 7.5 MB/s 
     |████████████████████████████████| 1.9 MB 44.5 MB/s 
     |████████████████████████████████| 1.5 MB 40.4 MB/s 
     |████████████████████████████████| 3.3 MB 43.9 MB/s 
     |████████████████████████████████| 9.5 MB 19.0 MB/s 
     |████████████████████████████████| 22.3 MB 55.2 MB/s 
     |████████████████████████████████| 31.0 MB 232 kB/s 
     |████████████████████████████████| 99 kB 7.8 MB/s 
     |████████████████████████████████| 14.1 MB 10.1 MB/s 
     |████████████████████████████████| 517 kB 42.3 MB/s 
     |████████████████████████████████| 60.5 MB 68 kB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 26.1 MB 1.2 MB/s 
     |████████████████████████████████| 298 kB 60.6 MB/s 
     |████████████████████████████████| 3.1 MB 35.1 MB/s 
     |█████████████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 7.1 MB/s 
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.2
    Uninstalling transformers-4.16.2:
      Successfully uninstalled transformers-4.16.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import torchaudio
import torch
import numpy as np
from transformers import AutoConfig, Wav2Vec2Processor, AutoModelForAudioClassification

In [11]:
model_name_or_path = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
pooling_mode = "mean"
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
target_sampling_rate = processor.feature_extractor.sampling_rate

Downloading:   0%|          | 0.00/262 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

In [12]:
def preprocess_function_eval(speech_path):   
    speech_array, sampling_rate = librosa.load(speech_path, sr = 16000)
    result = processor(speech_array, sampling_rate=target_sampling_rate, max_length=50000, padding=True, truncation=True, return_attention_mask=True)
    len_of_input_data = result['input_values'][0].shape[0]
    padded_array = np.pad(result['input_values'][0], ((0,50000-len_of_input_data)), constant_values=0)
    return padded_array

In [13]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id=class_to_idx,
    id2label=idx_to_class,
    finetuning_task="wav2vec2_clf",    
)
setattr(config, 'pooling_mode', pooling_mode)

In [14]:
model = AutoModelForAudioClassification.from_pretrained(
    '/content/drive/MyDrive/models/wav2vec2/hugging_face_w2v2', # <- insert folder with xml, config and bin
    num_labels=6,
    label2id=class_to_idx,
    id2label=idx_to_class,
) 


In [15]:
from openvino.runtime import Core

ie = Core()
onnx_model_path = "/content/drive/MyDrive/models/wav2vec2/wav2vec2.onnx" #<- check onnx file using openvino engine
model_onnx = ie.read_model(model=onnx_model_path)
model_onnx.reshape([1, 50000])
compiled_model_onnx = ie.compile_model(model=model_onnx, device_name="CPU")

In [17]:
import pathlib 

path = pathlib.Path('/content/drive/MyDrive/data/AudioVideo/Val_AFEW_audio')

emotions_list = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
audio_proba = [] 

for emotion_name in emotions_list: 
  print(emotion_name)
  audio_path = path/emotion_name
  for audio in audio_path.iterdir():
      output = compiled_model_onnx([np.expand_dims(preprocess_function_eval(audio), 0)])
      audio_proba.append(softmax([*output.values()][0][0]))
audio_proba = np.array(audio_proba)

Angry
Disgust
Fear
Happy
Neutral
Sad
Surprise


In [ ]:
import pickle
model_name = 'w2v2'
MODEL2EMOTIW_FEATURES=model_name+'_prob_audio_onnx.pickle' 

print(MODEL2EMOTIW_FEATURES)

with open(MODEL2EMOTIW_FEATURES, 'wb') as handle:
    pickle.dump(prob_audio, handle, protocol=pickle.HIGHEST_PROTOCOL)

w2v2_prob_audio_afew.pickle


## Soft Voting

In [ ]:
path_to_audio_proba = '/content/drive/MyDrive/data/w2v2_prob_audio_afew.pickle'

with open(path_to_audio_proba, "rb") as fh:
  audio_proba = p.load(fh)

In [27]:
weights = np.arange(0, 0.5, 0.01)

best_weights = [0, 0]
best_acc = 0


for a in weights:
    for l in weights:
        y_pred = []
        for i in range(0, len(linsvc_proba)):
            max_prob = np.argmax(a*audio_proba[i]+l*linsvc_proba[i])
            y_pred.append(max_prob)
        if metrics.accuracy_score(y_test_enet, np.array(y_pred)) > best_acc:
            best_acc = metrics.accuracy_score(y_test_enet, np.array(y_pred))
            best_weights[0] = a
            best_weights[1] = l
            print(best_acc, best_weights)

0.1671018276762402 [0.0, 0.0]
0.5926892950391645 [0.0, 0.01]
0.6135770234986945 [0.01, 0.01]
0.6214099216710183 [0.02, 0.01]
0.6266318537859008 [0.05, 0.03]
0.6292428198433421 [0.17, 0.1]


In [28]:
best_weights = [0, 0, 0.17, 0.1]

y_pred = []

for i in range(0, len(linsvc_proba)):
  y_pred.append(np.argmax(best_weights[2]*audio_proba[i]+best_weights[3]*linsvc_proba[i]))

print("Complete accuracy:",metrics.accuracy_score(y_test_enet, np.array(y_pred)))

Complete accuracy: 0.6292428198433421
